#### Python Pandas has been used to combine all the excel files into one single dataframe.
#### Some preprocessing steps will also be done via python only

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob # To concatenate all the csv files and create a single dataframe

In [3]:
folder_path = r"Y:\Data\Data Analytics by Google Capstone\Cyclistic Data\CSV Files"
csv_files = glob.glob(folder_path + r"/*.csv" )  # Returns a list

In [4]:
type(csv_files)

list

In [5]:
df_list = (pd.read_csv(file) for file in csv_files)

In [6]:
df =pd.concat( df_list,ignore_index=True)

In [7]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,954144C2F67B1932,classic_bike,2022-07-05 08:12:47,2022-07-05 08:24:32,Ashland Ave & Blackhawk St,13224,Kingsbury St & Kinzie St,KA1503000043,41.907066,-87.667252,41.889177,-87.638506,member
1,292E027607D218B6,classic_bike,2022-07-26 12:53:38,2022-07-26 12:55:31,Buckingham Fountain (Temp),15541,Michigan Ave & 8th St,623,41.869621,-87.623981,41.872773,-87.623981,casual
2,57765852588AD6E0,classic_bike,2022-07-03 13:58:49,2022-07-03 14:06:32,Buckingham Fountain (Temp),15541,Michigan Ave & 8th St,623,41.869621,-87.623981,41.872773,-87.623981,casual
3,B5B6BE44314590E6,classic_bike,2022-07-31 17:44:21,2022-07-31 18:42:50,Buckingham Fountain (Temp),15541,Woodlawn Ave & 55th St,TA1307000164,41.869621,-87.623981,41.795264,-87.596471,casual
4,A4C331F2A00E79E0,classic_bike,2022-07-13 19:49:06,2022-07-13 20:15:24,Wabash Ave & Grand Ave,TA1307000117,Sheffield Ave & Wellington Ave,TA1307000052,41.891466,-87.626761,41.936253,-87.652662,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779439,D7BBF4BCBB72DA32,classic_bike,30-06-2023 12:58,30-06-2023 13:41,Fairbanks Ct & Grand Ave,TA1305000003,California Ave & Milwaukee Ave,13084,41.891847,-87.620580,41.922695,-87.697153,casual
5779440,9A1685F9A39646CA,electric_bike,29-06-2023 19:56,29-06-2023 20:09,Fairbanks Ct & Grand Ave,TA1305000003,NaN,NaN,41.891970,-87.620198,41.890000,-87.610000,casual
5779441,CD4CC5A60881C7AF,electric_bike,25-06-2023 00:27,25-06-2023 00:39,Clark St & Lincoln Ave,13179,NaN,NaN,41.915745,-87.634604,41.920000,-87.650000,casual
5779442,FF6594685CFE2056,electric_bike,24-06-2023 21:26,24-06-2023 21:28,Fairbanks Ct & Grand Ave,TA1305000003,NaN,NaN,41.891725,-87.620607,41.890000,-87.620000,casual


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779444 entries, 0 to 5779443
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 573.2+ MB


Dropping the ride_id as it will not be of any help for visualizations

In [9]:
df.drop('ride_id', inplace=True, axis=1)

In [10]:
df.isnull().sum()

rideable_type              0
started_at                 0
ended_at                   0
start_station_name    857860
start_station_id      857992
end_station_name      915655
end_station_id        915796
start_lat                  0
start_lng                  0
end_lat                 5795
end_lng                 5795
member_casual              0
dtype: int64

There are a lot of null values in the following features-
- start_station_name    857860
- start_station_id      857992
- end_station_name      915655
- end_station_id        915796

As can be seen, there are observations where start_station_name is not null but start_station_id is null.

There are observations in the dataset where the same start_station names appear and there are station_id values present agaisnt it. 

Thus the null values can be replaced by those station ids

In [11]:
# Creating a function for the purpose
def names_to_id(id,naam):
    # Unique value of start staion names where station names exist but station ids do not
    name_replacements = df[(naam.notnull() ) & (id.isnull())][naam.name].unique() # naam.name to get the name of the column as id represents the entire series
    names_id = df[(naam.isin(name_replacements)) & id.notnull()][[id.name,naam.name]].drop_duplicates()
    names_id_dict = dict(zip(names_id[id.name],names_id[naam.name]))
    #print(names_id_dict)
    return df[naam.name].map(names_id_dict).fillna(df[id.name])


In [12]:
df['start_station_id']= names_to_id(df['start_station_id'],df['start_station_name'])

In [13]:
df['end_station_id']= names_to_id(df['end_station_id'],df['end_station_name'])

In [20]:
df.isnull().sum()

rideable_type              0
started_at                 0
ended_at                   0
start_station_name    857860
start_station_id      857992
end_station_name      915655
end_station_id        915796
start_lat                  0
start_lng                  0
end_lat                 5795
end_lng                 5795
member_casual              0
dtype: int64

Now there are as many null values against the start_station_id as are against start_station_name

Performing similar opertaion for end stations as well

In [26]:
df.isnull().sum()

rideable_type              0
started_at                 0
ended_at                   0
start_station_name    857860
start_station_id      857992
end_station_name      915655
end_station_id        915796
start_lat                  0
start_lng                  0
end_lat                 5795
end_lng                 5795
member_casual              0
dtype: int64

Again there are as many null values against the end_station_id as are against end_station_name

Trying to impute the null values by checking if latitude and longitude values corresponding to certain observations are same, then their station_id and station_name (both start and end) will also be same.


In [27]:
df[df['start_station_name'].isnull()]

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
1719,electric_bike,2022-07-04 15:04:26,2022-07-04 15:32:38,NaN,NaN,Ashland Ave & Blackhawk St,13224,41.95,-87.64,41.907066,-87.667252,member
1720,electric_bike,2022-07-12 14:43:51,2022-07-12 14:49:28,NaN,NaN,Cornell Ave & Hyde Park Blvd,KA1503000007,41.80,-87.59,41.802406,-87.586924,member
1721,electric_bike,2022-07-10 11:37:27,2022-07-10 11:40:03,NaN,NaN,Cornell Ave & Hyde Park Blvd,KA1503000007,41.80,-87.59,41.802406,-87.586924,member
2988,electric_bike,2022-07-23 07:41:53,2022-07-23 08:02:14,NaN,NaN,Brandon Ave & 91st St,847,41.74,-87.55,41.730000,-87.550000,member
2989,electric_bike,2022-07-30 07:26:54,2022-07-30 07:30:36,NaN,NaN,Public Rack - Chase Ave & Touhy Ave - NE,482,42.02,-87.69,42.010000,-87.690000,member
...,...,...,...,...,...,...,...,...,...,...,...,...
5774141,electric_bike,19-06-2023 21:55,19-06-2023 22:10,NaN,NaN,NaN,NaN,41.97,-87.65,41.940000,-87.660000,casual
5774150,electric_bike,08-06-2023 00:02,08-06-2023 00:12,NaN,NaN,NaN,NaN,41.93,-87.69,41.930000,-87.710000,casual
5774151,electric_bike,07-06-2023 20:35,07-06-2023 20:59,NaN,NaN,NaN,NaN,41.85,-87.61,41.880000,-87.650000,casual
5774152,electric_bike,08-06-2023 17:22,08-06-2023 17:27,NaN,NaN,NaN,NaN,41.94,-87.67,41.940000,-87.650000,casual


In [28]:
df[(df['end_lat']== 41.95) & (df['end_lng']== -87.64) & (df['end_station_id'].notnull())]

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual


As can be seen, there are no such observations where latitude and longitude values are same and corresponding station id and station names are not null

Thus this method of imputation cannot be implemented

We can impute by using mode- but this will create a bias as there are a lot of null values and imputing all of those by a single value will create a bias

In [29]:
df['start_station_name'].mode().values[0]

'Streeter Dr & Grand Ave'

Trying to impute by choosing values randomly

In [30]:
# Function for random imputation
def random_imputation(x): # x is the dataframe column containing null values
    null_num = x.isnull().sum() # No. of null values
    random_values = x.dropna().sample(null_num,random_state=4) # Creating a series that contains values taken randomly from start_station_id of the df
    # print(random_values)
    random_values.index = df[x.isnull()].index # Assigning the same indices to this series containing random values as are the indices of null values in the column of df
    return x.fillna(random_values)

In [31]:
df['start_station_id'] = random_imputation(df['start_station_id']) # The null values of the start_station_id of the dataframe are replaced by the values of newly created df

In [32]:
df['end_station_id'] = random_imputation(df['end_station_id']) # The null values of the end_station_id of the dataframe are replaced by the values of newly created df

In [33]:
df.isnull().sum()

rideable_type              0
started_at                 0
ended_at                   0
start_station_name    857860
start_station_id           0
end_station_name      915655
end_station_id             0
start_lat                  0
start_lng                  0
end_lat                 5795
end_lng                 5795
member_casual              0
dtype: int64

Imputing station_names(start and end) by checking values of station_id(start and end)

In [34]:
# Creating a function for the purpose
def id_to_names(id,naam):
    #print(id.name)
    id_replacements = df[(id.notnull()) & (naam.isnull())][id.name].unique() # id.name to get the name of the column as id represents the entire series
    id_names = df[(id.isin(id_replacements)) & naam.notnull()][[id.name,naam.name]].drop_duplicates()
    #print(id_names)
    id_names_dict = dict(zip(id_names[id.name],id_names[naam.name]))
    return df[id.name].map(id_names_dict).fillna(df[naam.name])

In [35]:
df['start_station_name'] = id_to_names(df['start_station_id'], df['start_station_name'])

In [36]:
df['end_station_name'] = id_to_names(df['end_station_id'], df['end_station_name'])

In [37]:
df.isnull().sum()

rideable_type            0
started_at               0
ended_at                 0
start_station_name       0
start_station_id         0
end_station_name         0
end_station_id           0
start_lat                0
start_lng                0
end_lat               5795
end_lng               5795
member_casual            0
dtype: int64

In [42]:
df.isnull().sum()

rideable_type            0
started_at               0
ended_at                 0
start_station_name       0
start_station_id         0
end_station_name         0
end_station_id           0
start_lat                0
start_lng                0
end_lat               5795
end_lng               5795
member_casual            0
dtype: int64

Now only end_lat and end_lang values have null values in them

In [43]:
null_lat_long = df[(df['end_lat'].isnull()) & (df['end_lng'].isnull())]['end_station_id'].unique()
len(null_lat_long)

591

In [44]:
station_to_lat_lng = df[df['end_station_id'].isin(null_lat_long) & df['end_lat'].notnull()][['end_station_id','end_lat', 'end_lng']].drop_duplicates()
station_to_lat_lng

,end_station_id,end_lat,end_lng
0,KA1503000043,41.889177,-87.638506
1,623,41.872773,-87.623981
3,TA1307000164,41.795264,-87.596471
4,TA1307000052,41.936253,-87.652662
5,WL-008,41.867118,-87.641088
...,...,...,...
5779240,13258,41.960000,-87.770000
5779252,18067,41.910000,-87.740000
5779282,13271,41.900000,-87.710000
5779370,TA1305000030,41.780000,-87.690000


In [45]:
lat_lng_dict = dict(zip(station_to_lat_lng['end_station_id'],zip(station_to_lat_lng['end_lat'],station_to_lat_lng['end_lng'])))
#lattitude and longitude have been zipped together and the tuple thus created is the value for end station id which acts as a key
print(lat_lng_dict)

{'KA1503000043': (41.82, -87.63), '623': (41.9, -87.61), 'TA1307000164': (41.83, -87.64), 'TA1307000052': (41.75, -87.65), 'WL-008': (41.94, -87.78), 'TA1307000128': (41.94, -87.74), 'TA1305000022': (41.74, -87.73), '15529': (41.8, -87.72), 'KA150400009X': (41.97, -87.71), '13081': (41.86, -87.61), 'TA1307000039': (42.0, -87.82), 'TA1305000001': (41.82, -87.6), '620': (41.86, -87.69), 'TA1306000010': (41.68, -87.64), '13022': (41.75, -87.74), 'LF-005': (41.98, -87.84), '15544': (41.95, -87.81), '13263': (41.74, -87.71), '632': (42.07, -87.69), 'TA1307000161': (41.95, -87.74), '13059': (41.81, -87.68), 'KA1503000074': (41.82, -87.62), '13063': (41.88, -87.72), 'Hubbard Bike-checking (LBS-WH-TEST)': (41.91, -87.68), '13257': (41.78, -87.76), '13294': (41.84, -87.66), 'KA1503000007': (41.85, -87.69), '13224': (42.05, -87.69), 'TA1307000115': (41.86, -87.64), '661': (41.9, -87.69), 'KA1504000134': (41.73, -87.74), '13074': (41.7, -87.66), '17660': (41.92, -87.77), '13194': (41.98, -87.71),

In [46]:
lat_lng_dict['KA1503000043']

(41.82, -87.63)

In [47]:
lat_lng_dict['KA1503000043'][1]

-87.63

In [59]:
df['end_lat'] = df['end_station_id'].map(lat_lng_dict).apply(lambda x: x[0] if isinstance(x, tuple) else x).fillna(df['end_lat'])

In [56]:
df['end_lng'] = df['end_station_id'].map(lat_lng_dict).apply(lambda x: x[1] if isinstance(x, tuple) else x).fillna(df['end_lng'])

In [57]:
df

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,classic_bike,2022-07-05 08:12:47,2022-07-05 08:24:32,Ashland Ave & Blackhawk St,13224,Kingsbury St & Kinzie St,KA1503000043,41.907066,-87.667252,41.82,-87.63,member
1,classic_bike,2022-07-26 12:53:38,2022-07-26 12:55:31,Buckingham Fountain,15541,Elizabeth St & Randolph St,623,41.869621,-87.623981,41.90,-87.61,casual
2,classic_bike,2022-07-03 13:58:49,2022-07-03 14:06:32,Buckingham Fountain,15541,Elizabeth St & Randolph St,623,41.869621,-87.623981,41.90,-87.61,casual
3,classic_bike,2022-07-31 17:44:21,2022-07-31 18:42:50,Buckingham Fountain,15541,Elizabeth St & Randolph St,TA1307000164,41.869621,-87.623981,41.83,-87.64,casual
4,classic_bike,2022-07-13 19:49:06,2022-07-13 20:15:24,Elizabeth St & Randolph St,TA1307000117,Sheffield Ave & Wellington Ave,TA1307000052,41.891466,-87.626761,41.75,-87.65,member
...,...,...,...,...,...,...,...,...,...,...,...,...
5779439,classic_bike,30-06-2023 12:58,30-06-2023 13:41,Elizabeth St & Randolph St,TA1305000003,Elizabeth St & Randolph St,13084,41.891847,-87.620580,41.79,-87.77,casual
5779440,electric_bike,29-06-2023 19:56,29-06-2023 20:09,Elizabeth St & Randolph St,TA1305000003,Kedzie Ave & Palmer Ct,13292,41.891970,-87.620198,41.85,-87.69,casual
5779441,electric_bike,25-06-2023 00:27,25-06-2023 00:39,Clark St & Lincoln Ave,13179,Elizabeth St & Randolph St,LF-005,41.915745,-87.634604,41.98,-87.84,casual
5779442,electric_bike,24-06-2023 21:26,24-06-2023 21:28,Elizabeth St & Randolph St,TA1305000003,LaSalle St & Jackson Blvd,TA1309000004,41.891725,-87.620607,41.98,-87.76,casual


In [58]:
df.isnull().sum()

rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

Thus all the null values have been imputed

Calculating ride length as end time -start time

In [66]:
df['ended_at'] = pd.to_datetime(df['ended_at']).astype('datetime64')

In [67]:
df['started_at'] = pd.to_datetime(df['started_at']).astype('datetime64')

In [69]:
df['ride_length'] = df['ended_at']- df['started_at']

Checking for day of the week on which a particular ride took place

In [70]:
df['day_of_week'] = df['started_at'].dt.dayofweek + 1 # 1 is added to make sunday as 1

Checking for monthn in which the ride took place

In [71]:
df['month'] = df['started_at'].dt.month

In [72]:
df

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,month
0,classic_bike,2022-07-05 08:12:47,2022-07-05 08:24:32,Ashland Ave & Blackhawk St,13224,Kingsbury St & Kinzie St,KA1503000043,41.907066,-87.667252,41.82,-87.63,member,0 days 00:11:45,2,7
1,classic_bike,2022-07-26 12:53:38,2022-07-26 12:55:31,Buckingham Fountain,15541,Elizabeth St & Randolph St,623,41.869621,-87.623981,41.90,-87.61,casual,0 days 00:01:53,2,7
2,classic_bike,2022-07-03 13:58:49,2022-07-03 14:06:32,Buckingham Fountain,15541,Elizabeth St & Randolph St,623,41.869621,-87.623981,41.90,-87.61,casual,0 days 00:07:43,7,7
3,classic_bike,2022-07-31 17:44:21,2022-07-31 18:42:50,Buckingham Fountain,15541,Elizabeth St & Randolph St,TA1307000164,41.869621,-87.623981,41.83,-87.64,casual,0 days 00:58:29,7,7
4,classic_bike,2022-07-13 19:49:06,2022-07-13 20:15:24,Elizabeth St & Randolph St,TA1307000117,Sheffield Ave & Wellington Ave,TA1307000052,41.891466,-87.626761,41.75,-87.65,member,0 days 00:26:18,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779439,classic_bike,2023-06-30 12:58:00,2023-06-30 13:41:00,Elizabeth St & Randolph St,TA1305000003,Elizabeth St & Randolph St,13084,41.891847,-87.620580,41.79,-87.77,casual,0 days 00:43:00,5,6
5779440,electric_bike,2023-06-29 19:56:00,2023-06-29 20:09:00,Elizabeth St & Randolph St,TA1305000003,Kedzie Ave & Palmer Ct,13292,41.891970,-87.620198,41.85,-87.69,casual,0 days 00:13:00,4,6
5779441,electric_bike,2023-06-25 00:27:00,2023-06-25 00:39:00,Clark St & Lincoln Ave,13179,Elizabeth St & Randolph St,LF-005,41.915745,-87.634604,41.98,-87.84,casual,0 days 00:12:00,7,6
5779442,electric_bike,2023-06-24 21:26:00,2023-06-24 21:28:00,Elizabeth St & Randolph St,TA1305000003,LaSalle St & Jackson Blvd,TA1309000004,41.891725,-87.620607,41.98,-87.76,casual,0 days 00:02:00,6,6


id of stations would not be required now for visualizations. Thus dropping start_station_id and end_station_id

In [76]:
df.drop(['start_station_id','end_station_id'], axis=1, inplace = True)

In [77]:
df

,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,month
0,classic_bike,2022-07-05 08:12:47,2022-07-05 08:24:32,Ashland Ave & Blackhawk St,Kingsbury St & Kinzie St,41.907066,-87.667252,41.82,-87.63,member,0 days 00:11:45,2,7
1,classic_bike,2022-07-26 12:53:38,2022-07-26 12:55:31,Buckingham Fountain,Elizabeth St & Randolph St,41.869621,-87.623981,41.90,-87.61,casual,0 days 00:01:53,2,7
2,classic_bike,2022-07-03 13:58:49,2022-07-03 14:06:32,Buckingham Fountain,Elizabeth St & Randolph St,41.869621,-87.623981,41.90,-87.61,casual,0 days 00:07:43,7,7
3,classic_bike,2022-07-31 17:44:21,2022-07-31 18:42:50,Buckingham Fountain,Elizabeth St & Randolph St,41.869621,-87.623981,41.83,-87.64,casual,0 days 00:58:29,7,7
4,classic_bike,2022-07-13 19:49:06,2022-07-13 20:15:24,Elizabeth St & Randolph St,Sheffield Ave & Wellington Ave,41.891466,-87.626761,41.75,-87.65,member,0 days 00:26:18,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779439,classic_bike,2023-06-30 12:58:00,2023-06-30 13:41:00,Elizabeth St & Randolph St,Elizabeth St & Randolph St,41.891847,-87.620580,41.79,-87.77,casual,0 days 00:43:00,5,6
5779440,electric_bike,2023-06-29 19:56:00,2023-06-29 20:09:00,Elizabeth St & Randolph St,Kedzie Ave & Palmer Ct,41.891970,-87.620198,41.85,-87.69,casual,0 days 00:13:00,4,6
5779441,electric_bike,2023-06-25 00:27:00,2023-06-25 00:39:00,Clark St & Lincoln Ave,Elizabeth St & Randolph St,41.915745,-87.634604,41.98,-87.84,casual,0 days 00:12:00,7,6
5779442,electric_bike,2023-06-24 21:26:00,2023-06-24 21:28:00,Elizabeth St & Randolph St,LaSalle St & Jackson Blvd,41.891725,-87.620607,41.98,-87.76,casual,0 days 00:02:00,6,6


Saving this df as a csv file

In [78]:
df.to_csv('Combined Cyclist Data')